# Data Analysis with Jupyter Notebooks.

# Tutorial 6

Benjamin J. Morgan, University of Bath.

# Contents

- [Putting it all together](#all_together)

## Putting it all together – Fitting experimental data<a id='all_together'></a>

In your Key Skills Excel practical you used linear regression to analyse equilibrium constant data for the equilibrium between NO<sub>2</sub> and N<sub>2</sub>O<sub>4</sub>, to find $\Delta H_\mathrm{r}$ and $\Delta S_\mathrm{r}$ for this reaction.  

As an example of how the code skills you have learned can be combined to solve a chemical problem, using a Jupyter notebook, we can work through the same process here. In addition to revisiting various ideas from earlier in the tutorial, we will also see how to read and write data files.

#### Theory

The equilibrium reaction we have data for is

\begin{equation}
2\mathrm{NO}_2 \mathrm{(g)}\leftrightharpoons \mathrm{N}_2\mathrm{O}_4 \mathrm{(g)}
\end{equation}

Taking the equations relating $\Delta G$ to $K$ and to $\left\{\Delta H, \Delta S\right\}$:

\begin{equation}
\Delta G = -RT \ln K, \tag{1}
\end{equation}

\begin{equation}
\Delta G = \Delta H - T\Delta S; \tag{2}
\end{equation}

we get

\begin{equation}
\ln K = \frac{\Delta H}{RT}-\frac{\Delta S}{R}. \tag{3}
\end{equation}

This is in the form

\begin{equation}
y = mx + c
\end{equation}

\begin{equation}
\ln K = \frac{\Delta H}{R}\frac{1}{T} - \frac{\Delta S}{R}. \tag{4}
\end{equation}

and plotting $\ln(K)$ against $\frac{1}{T}$ should give a straight line, with slope $\frac{\Delta H}{R}$ and intercept $-\frac{\Delta S}{R}$.

## Setting up the notebook

Throughout this example we are going to be using `numpy`, `pandas` and `matplotlib`, so we `import` each module at the start of the notebook, and set `matplotlib` to show inline figures.

>```python
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
```

#### Analysis

The data from this experiment are stored in a text file in `data/equilbirium_constant.dat`, which looks like

```
# equilibrium constant data for 2 NO2 => N2O4  
# columns are: temperature (degrees Celsius), K
  
9   34.3
20  12
25  8.79
33  4.4
40  2.8
52  1.4
60  0.751
70  0.4
```

To read this dataset into the notebook we use `read_csv()` with specfic arguements to handle the whitespace column separation, the comments, and the column labels. 

>```python
data = pd.read_csv( 'data/equilibrium_constant.dat', 
                     delim_whitespace = True, 
                     comment='#', 
                     names = [ 'T (C)', 'K' ] )
data                     
```

Looking at `data` shows us 8 data points (numbered 0 to 7), where each has a temperature (in the **T (C)** column) and a measured equilibrium constant (in the **K** column).

Before we can plot these data, we need to convert the temperature to Kelvin, and calculate $\ln K$.

Remember that we can access a column in a DataFrame by using its label:

>```python
data['T (C)']
```

and can use this to create a *new* column to store the temperature in Kelvin.

>```python
data['T (K)'] = data['T (C)'] + 273.0
data
```

This has created a new column, with the label **T (K)**.

Next, we do the same to calculate a set of $\ln K$ values:

>```python
data['ln K'] = np.log( data['K'] )
data
```

We are now ready to plot $\ln K$ versus $1/T$.

>```python
x = 1.0 / data['T (K)']
y = data['ln K']
plt.plot( x, y, 'o' )
plt.xlabel( '1/T' )
plt.ylabel( 'ln K' )
plt.show()
```

Notice that this code calculates all the inverse temperatures in place. An alternative way to do this would be to generate a new column in `data` with all the $1/T$ values, and then plot this directly.

Using `linregress()` from `scipy.stats` we can find the slope and intercept for the line of best-fit.

>```python
from scipy.stats import linregress
slope, intercept, rvalue, pvalue, stderr = linregress( x, y )
print( "slope =", slope )
print( "intercept =", intercept )
```

Now that we know our data analysis involves importing `linregress` from `scipy.stats` we would usually move this line to the topmost code cell with the other `import` statements. This lets someone see what libraries are needed just by looking at the top of a notebook.

<div class="alert alert-success">
Delete the import statement from the previous code cell, and add it into the topmost code cell with the other import statements. Then re-run the entire notebook up to here by selecting Cell > Run All from the Menu.
</div>

To plot the best-fit line against the original data, we generate a new data set according to $y=mx+c$, with $m$ and $c$ as the slope and intercept from `linregress`.

>```python
y_fit = slope * x + intercept # remember, x is an array storing 1.0 / data['T (K)']
plt.plot( x, y, 'o' )
plt.plot( x, y_fit, '-' )
plt.xlabel( '1/T' )
plt.ylabel( 'ln K' )
plt.show()
```


Because we have calculated the slope and intercept, we can now calculate $\Delta H$ and $\Delta S$ for the reaction.

\begin{equation}
\Delta H = R \times \mathrm{slope}
\end{equation}

\begin{equation}
\Delta S = R \times \mathrm{intercept}
\end{equation}

To save us having to look up and type in the gas constant, $R$, we can use [`scipy.constants`](https://docs.scipy.org/doc/scipy-0.18.1/reference/constants.html#).

>```python
from scipy.constants import R # gas constant in J K^-1 mol^-1
print( R ) 
```

>```python
delta_H = R * slope
delta_S = R * intercept
print( 'Delta H =', delta_H / 1000, 'kJ mol^-1' )
print( 'Delta S = ', delta_S, 'J K^-1 mol^-1' )
```

To finish, although we have not done any particularly complicated analysis on the original data, we save our new data set.

Our modified data set is stored as a `pandas` DataFrame in `data`

>```python
data
```

To save this out to another file we can use [`DataFrame.to_csv()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_csv.html).

>```python
filename = 'data/modified_equilibrium_data.dat'
data.to_csv( filename, sep=' ', index=False)
```

`sep=' '` tells `to_csv()` to use spaces instead of commas to separate the columns.

which saves the complete modified data set in plain text to `modified_equilibrium_data.dat` in the `data` directory:

```
"T (C)" K "T (K)" "ln K"
9 34.3 282.0 3.535145354171894
20 12.0 293.0 2.4849066497880004
25 8.79 298.0 2.1736147116970854
33 4.4 306.0 1.4816045409242156
40 2.8 313.0 1.0296194171811581
52 1.4 325.0 0.3364722366212129
60 0.7509999999999999 333.0 -0.28634962721800244
70 0.4 343.0 -0.916290731874155
```

Or for a cleaner saved file, we can use `np.round()` to round off the data to our required number of decimal places, before using `to_csv()`:

>```python
np.round(data, 3).to_csv( filename, sep=' ', index=False)
```